In [1]:
# ============================================================================
# CloakID Phase 2: Verification Notebook
# Robustness Testing against Instruction-based Diffusion Models
# ============================================================================

# %% — Cell 1: Setup & Installs
# ============================================================================
import subprocess, sys

def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

print(" Installing AI Generation libraries...")
for p in ["diffusers", "transformers", "accelerate", "gradio", "torch", "numpy", "pillow"]:
    install(p)

print(" Dependencies installed.")

# %% — Cell 2: Load Attacker Model
# ============================================================================
import torch
from diffusers import StableDiffusionInstructPix2PixPipeline, EulerAncestralDiscreteScheduler
from PIL import Image
import numpy as np
import gradio as gr
import warnings

warnings.filterwarnings("ignore")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float16 if DEVICE.type == "cuda" else torch.float32

print(f"  Running on {DEVICE} ({DTYPE})")

print(" Loading Attacker Model (timbrooks/instruct-pix2pix)...")
# We use InstructPix2Pix because it represents the "Text-Guided Manipulation" threat
# effectively. It tries to keep the original structure while following the prompt.
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    "timbrooks/instruct-pix2pix",
    torch_dtype=DTYPE,
    safety_checker=None
).to(DEVICE)

# Use Euler Ancestral scheduler for fast, creative edits
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

print(" Attacker Model Ready.")

# %% — Cell 3: Attack Logic
# ============================================================================

def resize_for_ai(img, size=512):
    """Resize image to standard AI resolution for inference."""
    return img.convert("RGB").resize((size, size), Image.LANCZOS)

def simulate_attack(
    original_image: Image.Image,
    protected_image: Image.Image,
    instruction: str,
    steps: int = 30,
    text_guidance: float = 7.5,
    image_guidance: float = 1.5,
    seed: int = 42,
    progress=gr.Progress(track_tqdm=True)
):
    """
    Runs the AI editing process on both images simultaneously using the same seed.
    """
    if original_image is None or protected_image is None:
        raise gr.Error("Please upload BOTH the Original and Protected images.")

    # 1. Preprocess
    # We resize to 512x512 because that's the native resolution of the attacker model.
    # A robust defense (from Phase 1) must survive this resizing.
    img_orig = resize_for_ai(original_image)
    img_prot = resize_for_ai(protected_image)

    # 2. Setup Generator (Deterministic for fair comparison)
    generator = torch.Generator(DEVICE).manual_seed(int(seed))

    print(f" Executing Attack: '{instruction}'")

    # 3. Attack Original (Control Group)
    result_orig = pipe(
        prompt=instruction,
        image=img_orig,
        num_inference_steps=steps,
        image_guidance_scale=image_guidance,
        guidance_scale=text_guidance,
        generator=generator
    ).images[0]

    # Reset generator for exact comparison
    generator = torch.Generator(DEVICE).manual_seed(int(seed))

    # 4. Attack Protected (Experimental Group)
    result_prot = pipe(
        prompt=instruction,
        image=img_prot,
        num_inference_steps=steps,
        image_guidance_scale=image_guidance,
        guidance_scale=text_guidance,
        generator=generator
    ).images[0]

    return result_orig, result_prot

# %% — Cell 4: Verification Dashboard (Gradio)
# ============================================================================

css = """
.gradio-container {background-color: #1e1e2e}
h1, h2, h3, p {color: #cdd6f4}
"""

with gr.Blocks(title="CloakID Verification", theme=gr.themes.Soft(), css=css) as demo:
    gr.Markdown("#  CloakID Phase 2: Robustness Verification")
    gr.Markdown(
        "Upload the **Original Image** and the **Protected Image** (from Phase 1). "
        "Enter a malicious prompt to see if the defense prevents the AI from editing the identity."
    )

    with gr.Row():
        # Input Column
        with gr.Column(scale=1):
            gr.Markdown("### 1. Upload Targets")
            with gr.Row():
                in_orig = gr.Image(label="Original Image (Unprotected)", type="pil", height=250)
                in_prot = gr.Image(label="Protected Image (Immunized)", type="pil", height=250)

            gr.Markdown("### 2. Configure Attack")
            instruction = gr.Textbox(
                label="Malicious Instruction",
                value="change the backround color",
                placeholder="e.g., make him wear a suit, Make him smile..."
            )
            
            with gr.Accordion("Advanced Attack Parameters", open=False):
                seed = gr.Number(label="Random Seed", value=1234, precision=0)
                steps = gr.Slider(10, 100, value=30, label="Inference Steps")
                text_cfg = gr.Slider(1.0, 20.0, value=7.5, label="Text Guidance (Prompt Strength)")
                img_cfg = gr.Slider(1.0, 5.0, value=1.5, label="Image Guidance (Structure Preservation)")

            attack_btn = gr.Button(" Generate image", variant="stop", size="lg")

        # Output Column
        with gr.Column(scale=1):
            gr.Markdown("### 3. Simulation Results")
            with gr.Row():
                out_orig = gr.Image(label="Result on Original (Should Succeed)", type="pil")
                out_prot = gr.Image(label="Result on Protected (Should Fail)", type="pil")
    
    # Wire Logic
    attack_btn.click(
        fn=simulate_attack,
        inputs=[in_orig, in_prot, instruction, steps, text_cfg, img_cfg, seed],
        outputs=[out_orig, out_prot]
    )

    gr.Markdown("### How to interpret results:")
    gr.Markdown(
        "- **Left Image:** If this looks like a high-quality edit, the model works correctly.\n"
        "- **Right Image:** If this looks like **gray noise, a blurry blob, or unrelated colors**, "
        "then **CloakID Protection is SUCCESSFUL**."
    )

# %% — Cell 5: Launch
# ============================================================================
print(" Launching Verification Dashboard...")
demo.queue().launch(share=True, debug=True)

 Installing AI Generation libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.5 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.22.1 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
langchain-core 0.3.79 requires packaging<26.0.0,>=23.2.0, but you have packaging 26.0rc2 which is incompatible.
fastai 2.8.4 requires fastcore<1.9,>=1.8.0, but you have fastcore 1.11.3 which is incompatible.


 Dependencies installed.


2026-02-13 09:19:14.919889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770974355.099500      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770974355.149401      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770974355.568308      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770974355.568342      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770974355.568345      55 computation_placer.cc:177] computation placer alr

  Running on cuda (torch.float16)
 Loading Attacker Model (timbrooks/instruct-pix2pix)...


...

  0%|          | 0/30 [00:00<?, ?steps/s]

 Executing Attack: 'add jackie chan near to him'


  0%|          | 0/30 [00:00<?, ?steps/s]

  0%|          | 0/30 [00:00<?, ?steps/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c87e370a7a2f64226a.gradio.live
